In [1]:
from doctest import testmod

# Chap. 11 --- Arbres binaires de recherche

Imaginons une bibliothèque qui contienne vraiment beaucoup de livres, répartis dans 17576 salles reliées les unes aux autres par des portes. Chaque salle contient une porte d'entrée et, possiblement, deux portes de sorties vers deux autres salles. Pour retrouver facilement l'emplacement d'un livre, les bibliothécaires astucieux ont eu l'idée suivante. 

Dans la toute première salle, dans laquelle on entre par la porte d'entrée de la bibliothèque, ils ont mis tous les ouvrages dont le titre commence par NAA. Si en revanche le titre commence par trois lettres venant avant NAA dans l'ordre alphabétique, alors il faut prendre la porte de sortie de gauche. Sinon, il faut prendre la porte
de sortie de droite. On continue alors la recherche dans la salle suivante. Si on est allé dans la salle de gauche, par exemple, on y trouve tous les ouvrages dont le titre commence par GNA. Là, de même, si le titre commence par trois lettres venant avant GNA dans l'ordre alphabétique, alors il faut prendre la porte de sortie de gauche, sinon la porte de droite. Et ainsi de suite jusqu'à ce qu'on parvienne à la salle contenant tous les livres dont les trois premières lettres du titre sont celles du titre qui nous intéresse. 

Une telle organisation est incroyablement efficace.

<div class="alert alert-warning" role="alert">

**Activité**

Combien de salles au maximum pour trouver un ouvrage ?
</div>

Bibliothèque de $17 576$ salles.

![](img02.png?1)

## 11.1 --- Notion

Un **arbre binaire de recherche** (abrégé en **ABR** dans la suite) est un arbre binaire dont les nœuds contiennent des valeurs
qui peuvent être comparées entre elles (par exemple des entiers ou des chaînes de caractères), et tel que pour tout nœud de l'arbre :

- **toutes les valeurs situées dans le sous-arbre gauche sont plus petites que la valeur située dans le nœud**,
- **toutes les valeurs situées dans le sous-arbre droit sont plus grandes que la valeur située dans le nœud**.



<div class="alert alert-warning" role="alert">

**Activité**

Quel(s) est(sont) les arbres binaires qui ne sont pas de recherche ?

![](img03.png)

</div>

### Représentation en Python

- classe `Noeud`
- valeurs comparables avec `<, <=, >, >=` (non codé)
- vérifie propriété d'ABR (non codé)

### Opérations

- `taille`
- `hauteur`
- `parcours_infixe`

<div class="alert alert-warning" role="alert">

**Activité**

Qu'affiche un parcours infixe sur un arbre binaire de recherche ?

</div>

## 11.2 --- Recherche dans un ABR

La propriété des ABR => recherche extrêmement efficace car à chaque étape, on élimine complètement la recherche dans un des sous arbres.

In [9]:
class Noeud:
    def __init__(self, g, v, d):
        self.gauche = g
        self.valeur = v 
        self.droit  = d

In [23]:
def appartient(x, a):
    if a is None:
        return False
    
    if x < a.valeur:
        return appartient(x, a.gauche)
    elif x > a.valeur:
        return appartient(x, a.droit)
    else:
        return True


In [26]:
a_1 = Noeud( Noeud (None, 1, Noeud(None, 2, None)), 3, Noeud(None, 4, None) )
a_2 = Noeud( Noeud(Noeud(None, 1, None), 2, None),  3, Noeud(None, 4, None) )

appartient(0, a_1)
appartient(3, a_1)

True

#### Efficacité

- Si ABR équitablement répartit : très efficace
- Pire cas : recherche infructueuse dans un peigne
- coût de la recherche majorée par la hauteur

## 11.3 --- Ajout dans un ABR

On fait le choix de considérer la classe `Noeud` comme immuable.
La fonction `ajoute` ne modifie donc pas un arbre, mais renvoie un *nouvel* arbre contenant `x` et tous les éléments de `a`.

In [29]:
def ajoute(x, a):
    if a is None:
        return Noeud(None, x, None)
    
    if x < a.valeur:
        return Noeud( ajoute(x, a.gauche) , a.valeur, a.droit)
    else:
        return Noeud(a.gauche, a.valeur, ajoute(x, a.droit) )
    

#### Efficacité

- ressemble à appartient (car même décision à chaque étape)
- majorée par la hauteur
- complexité mémoire comme compléxité temporelle car autant de noeud que d'appels récursifs

<div class="alert alert-warning" role="alert">

**Activité**

Détailler ce qui ce passe en mémoire lorsqu'on effectue la séquence suivante :

```python
a = ajoute(3, None)
a = ajoute(1, a)
a = ajoute(4, a)
```

</div>

<div class="alert alert-warning" role="alert">

**Activité**

Quel ABR est renvoyé si on ajoute à l'arbre précédent la valeur 3 ?

</div>

## 11.4 --- Suppression dans un ABR (HP)

In [31]:
def minimum(a):
    if a is None:
        return None
    if a.gauche is None:    # racine est minimum
        return a.valeur
    else:
        return minimum(a.gauche)

def supprime_minimum(a):
    assert a is not None
    if a.gauche is None:
        return a.droit
    return Noeud(supprime_minimum(a.gauche), a.valeur, a.droit)

def supprime(x, a):
    if a is None:
        return None

    if x < a.valeur:
        return Noeud(supprime(x, a.gauche), a.valeur, a.droit)
    elif x > a.valeur:
        return Noeud(a.gauche, a.valeur, supprime(x, a.droit))
    # supprimer la racine !
    elif a.droit is None:
        return a.gauche
    else:
        return Noeud(a.gauche, minimum(a.droit), supprime_minimum(a.droit))

## 11.5 --- Encapsulation dans un objet

In [33]:
class ABR:
    def __init__ (self):
        self.racine = None
    
    def ajouter(self, x):
        self.racine = ajoute(x, self.racine)
    
    def contient(self, x):
        return appartient(x, self.racine)

<div class="alert alert-warning" role="alert">

**Activité**

Représenter la mémoire après la séquence suivante :

```python
a = ABR()
a.ajouter(3)
a.ajouter(1)
a.ajouter(4)
a.ajouter(2)
```

</div>

![](img04.png)

## 11.6 --- Arbre équilibré

- il est possible de s'assurer que la hauteur d'un ABR ne soit pas trop grande
- arbres AVL
- arbres rouges/noirs

## 11.7 --- Activités

<div class="alert alert-warning" role="alert">

**Exercice 1**

Pourquoi la bibliothèque donnée en exemple au début de ce chapitre contient-elle 17576 salles?

</div>

Car il y a $26^3 = 17 576$ mots de trois lettres.

<div class="alert alert-warning" role="alert">

**Exercice 2** 

Donner tous les ABR formés de trois nœuds et contenant les entiers 1, 2 et 3.

</div>

![](img01.png)

<div class="alert alert-warning" role="alert">

**Exercice 3**

Dans un ABR, où se trouve le plus petit élément? En déduire une fonction `minimum(a)` qui renvoie le plus petit élément de l'ABR `a`. Si l'arbre `a` est vide, alors cette fonction renvoie `None`.

</div>

Par définition d'un ABR, le plus petit élément se trouve en bas à gauche de l'arbre.

In [1]:
def minimum(a):
    if a is None:
        return None

    if a.gauche is None:
        return a.valeur
    else:
        return minimum(a.gauche)

<div class="alert alert-warning" role="alert">

**Exercice 4**

Écrire une variante de la fonction `ajoute` du cours qui n'ajoute pas l'élément `x` à l'arbre `a` s'il est déjà dedans.

</div>

In [28]:
def ajoute(x, a):
    if a is None:
        return Noeud(None, x, None)
    
    if x < a.valeur:
        gauche = ajoute(x, a.gauche)
        return Noeud(gauche, a.valeur, a.droit)
    elif x > a.valeur:
        droit = ajoute(x, a.droit)
        return Noeud(a.gauche, a.valeur, droit)
    else:
        return a # car x est déjà dans a

<div class="alert alert-warning" role="alert">

**Exercice 5**

Écrire une fonction `compte(x, a)` qui renvoie le nombre d'occurrences de `x` dans l'ABR `a`. 

On ne suppose pas que l'arbre `a` a été construit à partir de la fonction `ajoute`, mais seulement qu'il s'agit d'un ABR. Cela veut dire qu'une valeur égale à la racine peut se trouver encore dans le sous-arbre gauche autant que dans le sous-arbre droit. Cela étant, on s'attachera à ne pas descendre dans les sous-arbres dans lesquels on est certain que la valeur `x` ne peut apparaître.

</div>

Lorsque `x` est strictement plus petit ou strictement plus grand que `a.valeur`, il suffit d'aller d'un seul côté. En cas d'égalité, en
revanche, il faut continuer à compter des deux côtés.

In [3]:
def compte(x, a):
    if a is None:
        return 0
    
    if x < a.valeur:
        return compte(x, a.gauche)
    elif x > a.valeur:
        return compte(x, a.droit)
    else:
        return 1 + compte(x, a.gauche) + compte(x, a.droit)

<div class="alert alert-warning" role="alert">

**Exercice 6**

Écrire une fonction `remplir(a, t)` qui ajoute tous les éléments
de l'arbre `a` dans le tableau `t`, dans l'ordre **infixe**.

Chaque élément `x` est ajouté au tableau `t` avec `t.append (x)`.

Ajouter ensuite une méthode `lister(self)` à la classe `ABR` qui renvoie un nouveau tableau contenant tous les éléments de l'ABR `self` par ordre croissant.

</div>

C'est un cas particulier de parcours infixe :

In [4]:
def remplir(a, t):
    if a is None:
        return
    
    remplir(a.gauche, t)
    t.append(a.valeur)
    remplir(a.droit, t)

Pour la méthode `lister`, on crée un nouveau tableau, que l'on remplit avec
la fonction `remplir`.

In [5]:
def lister(self):
    t = []
    remplir(self.racine, t)
    return t

<div class="alert alert-warning" role="alert">

**Exercice 7**

En utilisant l'exercice précédent, écrire une fonction `trier(t)` qui reçoit en argument un tableau d'entiers et renvoie un tableau trié contenant les mêmes éléments.

Quelle est l'efficacité de ce tri?

</div>

L'idée est d'ajouter tous les éléments du tableau `t` dans un ABR, puis d'utiliser la méthode `lister` développée exercice précédemment.

In [6]:
def trier(t):
    a = ABR()
    for x in t:
        a.ajouter(x)
    return a.lister()

L'efficacité de ce tri dépend fortement de la répartition des éléments dans
le tableau initial. Si par exemple les éléments sont déjà triés dans le tableau initial, alors l'arbre sera un peigne et le coût de la construction de l'ABR sera donc quadratique.